<a href="https://colab.research.google.com/github/gyuminpk/Playground_solver/blob/main/Lights_Out_Solver.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
#켜져있는 칸 좌표 입력
on_grids = [(0, 2), (1, 1), (2, 0), (2, 1), (2, 4), (3, 1), (3, 2), (4, 2), (4, 4)]

In [6]:
import copy

def toggle(board, r, c):
    """
    Toggles the state of (r, c) and its valid neighbors on the board.
    The board is modified in-place.
    """
    rows = len(board)
    cols = len(board[0])

    # Coordinates to toggle: (r, c) and its four cardinal neighbors
    neighbors_relative_coords = [(0, 0), (-1, 0), (1, 0), (0, -1), (0, 1)]

    for dr, dc in neighbors_relative_coords:
        nr, nc = r + dr, c + dc
        # Ensure the neighbor is within the 0-4 grid boundaries
        if 0 <= nr < rows and 0 <= nc < cols:
            board[nr][nc] = 1 - board[nr][nc] # Flip state (0 to 1, 1 to 0)

def solve_lights_out():
    grid_size = 5
    # Initialize the board with specific lights ON as per the problem description
    # (0,0), (1,1), (2,2), (3,3) are ON (represented by 1)
    initial_board = [[0 for _ in range(grid_size)] for _ in range(grid_size)]

    for r, c in on_grids:
        initial_board[r][c] = 1

    print("Initial Board State (1=ON, 0=OFF):")
    for row in initial_board:
        print(row)
    print("-" * 40)

    # Iterate through all 2^grid_size (2^5 = 32) possible click combinations for the first row.
    # Each integer 'i' from 0 to 31 represents a unique binary pattern for the first row clicks.
    # For example, if i=1 (binary 00001), only the last column of the first row is clicked.
    for i in range(2**grid_size):
        # Create a deep copy of the initial board and a new clicks tracking matrix for this attempt
        current_board = copy.deepcopy(initial_board)
        clicks_made = [[0 for _ in range(grid_size)] for _ in range(grid_size)]

        # Apply clicks for the first row based on the current combination 'i'
        for c in range(grid_size):
            if (i >> c) & 1:  # Check if the c-th bit is set in 'i' (meaning a click in column c)
                toggle(current_board, 0, c) # Apply the click effect to the board
                clicks_made[0][c] = 1       # Record the click

        # Propagate clicks for subsequent rows (from row 1 to row 4)
        # For each cell (r, c), if the light directly above it (r-1, c) is ON,
        # we *must* click (r, c) to turn off the light at (r-1, c).
        # This is a deterministic step to clear the lights row by row from top down.
        for r in range(1, grid_size):
            for c in range(grid_size):
                if current_board[r-1][c] == 1: # If the light at (r-1, c) is ON
                    toggle(current_board, r, c) # Click the current cell (r, c)
                    clicks_made[r][c] = 1       # Record the click

        # After all clicks are made based on the first row combination and propagation,
        # check if all lights are off in the final state of the board.
        is_solved = True
        for r in range(grid_size):
            for c in range(grid_size):
                if current_board[r][c] == 1:
                    is_solved = False
                    break
            if not is_solved:
                break

        # If a solution is found, print it and return.
        if is_solved:
            print("Solution Found!")
            print("Click Sequence (1 means click, 0 means no click at that position):")
            for row in clicks_made:
                print(row)

            # Optional verification step: Apply the found clicks to a fresh initial board
            # to ensure it results in an all-off board.
            verification_board = copy.deepcopy(initial_board)
            for r in range(grid_size):
                for c in range(grid_size):
                    if clicks_made[r][c] == 1:
                        toggle(verification_board, r, c)

            print("\nVerification: Final Board State after applying clicks (should be all zeros):")
            for row in verification_board:
                print(row)

            return clicks_made # Return the first found solution

    # If no solution is found after trying all first-row combinations
    print("No solution found for the given initial state and rules.")
    return None

# Execute the solver function
solve_lights_out()

Initial Board State (1=ON, 0=OFF):
[0, 0, 1, 0, 0]
[0, 1, 0, 0, 0]
[1, 1, 0, 0, 1]
[0, 1, 1, 0, 0]
[0, 0, 1, 0, 1]
----------------------------------------
Solution Found!
Click Sequence (1 means click, 0 means no click at that position):
[0, 0, 1, 0, 0]
[0, 1, 0, 1, 0]
[1, 0, 1, 1, 1]
[0, 0, 0, 0, 1]
[1, 1, 0, 0, 0]

Verification: Final Board State after applying clicks (should be all zeros):
[0, 0, 0, 0, 0]
[0, 0, 0, 0, 0]
[0, 0, 0, 0, 0]
[0, 0, 0, 0, 0]
[0, 0, 0, 0, 0]


[[0, 0, 1, 0, 0],
 [0, 1, 0, 1, 0],
 [1, 0, 1, 1, 1],
 [0, 0, 0, 0, 1],
 [1, 1, 0, 0, 0]]